In [ ]:
from pyciss.io import RingCube, PathManager, db_mapped_cubes

In [ ]:
pm = PathManager("N1467345444") 

# meta development

In [ ]:
import pkg_resources as pr

In [ ]:
with pr.resource_stream('pyciss', 'data/ring_resonances.csv') as f:
    resonances = pd.read_csv(f)
resonances.head()

In [ ]:
resonances.sort_values(by='Resonance (km)',ascending=False).head()

In [ ]:
from pyciss.meta import meta_df

In [ ]:
meta_df.head()

In [ ]:
meta_df.loc[pm._id]

In [ ]:
cube = RingCube(pm.cubepath)

In [ ]:
cube.meta.pixres

In [ ]:
%matplotlib nbagg

In [ ]:
cube.imshow()

In [ ]:
cube.meta

In [ ]:
dics = []
for p in db_mapped_cubes():
    d = {}
    d['path'] = p
    cube = RingCube(p)
    d['meta_res'] = cube.meta_pixres
    d['label_res'] = cube.resolution_val
    dics.append(d)

In [ ]:
df = pd.DataFrame(dics)

In [ ]:
df.head()

In [ ]:
df.plot(style='*-')

In [ ]:
cube = RingCube(df[df.label_res>5000].path.values[0])

In [ ]:
cube.imshow()

In [ ]:
cube.pm.id

In [ ]:
cube.get_opus_meta_data()

In [ ]:
cube.opusmeta.ring_geom